In [1]:
import pandas as pd

df_developed = pd.read_excel('车型验证表.xlsx', sheet_name='待验证车型表-已开发', dtype=str)
df_undeveloped = pd.read_excel('车型验证表.xlsx', sheet_name='待验证车型表-未开发', dtype=str)
df_special = pd.read_excel('车型验证表.xlsx', sheet_name='特殊车型匹配', dtype=str)
df_sales = pd.read_excel('车型验证表.xlsx', sheet_name='北美车型保有量', dtype=str)

In [2]:
def vehicle_(df, df_special):
    # 分离车型
    df['车型'] = df['车型'].apply(lambda x: x.split('\n'))
    df= df.explode('车型').fillna('')
    list_vehicle = df['车型'].tolist()
    ls_make_model = []
    ls_years = []
    for v in list_vehicle:
        list_split = v.split(' ')
        year = str(list_split[-1])
        make_model = ' '.join(list_split[:-1])
        years = []
        if '-' not in year:
            years.append(year)
        else:
            try:
                max_year = int(max(year.split('-')))
                min_year = int(min(year.split('-')))
                for y in range(min_year, max_year+1):
                    years.append(y)
            except:
                years.append('')
        ls_make_model.append(make_model)
        ls_years.append(years)
    df['Make Model'] = ls_make_model
    df['Year'] = ls_years
    df = df.explode('Year')
    df = df[['辨识号', 'Make Model', 'Year']].copy()
    
    # 匹配特殊车型
    df_match = pd.merge(df, df_special, left_on='Make Model', right_on='Match_Vehicle', how='left')
    df_match['make+model'] = df_match['make+model'].fillna(df_match['Make Model'])
    df_match = df_match[['辨识号', 'make+model', 'Year']]
    df_match['Year'] = pd.to_numeric(df_match['Year'], errors='coerce').astype('Int64')
    df_match['make+model'] = df_match['make+model'].str.title()
    df_match = df_match[df_match['Year'].notna()]

    return df_match

In [3]:
# 建立车型覆盖表
df_cover = df_sales.copy()
df_cover['Make+Model'] = df_cover['Make+Model'].str.title()
df_cover.set_index('Make+Model', inplace=True)
df_cover = df_cover.rename_axis(None)

# 行名
rows = df_cover.index.tolist()
# 列名
columns = df_cover.columns.tolist()

In [4]:
df_special['Match_Vehicle'] = df_special['make'] + ' ' + df_special['model']
# 已开发覆盖
df_match = vehicle_(df_developed, df_special)
for _, row in df_match.iterrows():
    row_val = row['make+model']
    col_val = row['Year']
    if row_val in rows and col_val in columns:
        df_cover.at[row_val, col_val] = '√'

In [5]:
# 未开发覆盖
df_match = vehicle_(df_undeveloped, df_special)
for _, row in df_match.iterrows():
    row_val = row['make+model']
    col_val = row['Year']
    if row_val in rows and col_val in columns:
        df_cover.at[row_val, col_val] = 'Sample'
df_cover = df_cover.replace('1', "×").replace('0', "-")

In [6]:
# 重置索引、输出
new_index = df_sales['Make+Model'].tolist()
df_cover.index = new_index
df_cover.to_excel('车型覆盖表.xlsx')
df_cover

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Ford F-Series,Data,Data,Data,Sample,Data,Data,Data,Data,Data,Data,...,Data,Data,Data,Data,Data,Data,Sample,Sample,Sample,Data
Chevrolet Silverado,Data,Sample,Sample,Sample,Sample,Sample,Sample,Sample,Sample,Sample,...,Data,Data,Data,Data,Sample,Sample,Sample,Data,Data,Data
Ram Pickup,√,√,×,×,×,×,×,×,×,×,...,Data,Data,Data,Sample,Sample,Data,Data,Data,Data,Data
Toyota Camry,Sample,Sample,√,Data,Data,Data,Data,Data,Data,Data,...,√,√,√,Data,Data,Data,Data,Data,Data,Data
Honda Accord,Sample,Sample,Sample,√,√,√,√,√,Data,Data,...,Data,Data,Data,Data,Data,Data,Data,Data,Data,Data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lamborghini Huracan,-,-,-,-,-,-,-,-,-,-,...,×,×,×,×,×,×,×,×,-,-
Audi Q8 E-Tron Sportback,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,○
BMW Xm,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,Data,Data
Rivian EDV 500,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,○,○
